In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the stock price data from the CSV file
data = pd.read_csv('stock_prices.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')
prices = data['StockPrice'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler()
prices_normalized = scaler.fit_transform(prices)

# Create sequences and labels for LSTM training
look_back = 30  # Number of previous days to use for predicting the next day's price
X, y = [], []
for i in range(len(prices_normalized) - look_back):
    X.append(prices_normalized[i:i+look_back, 0])
    y.append(prices_normalized[i+look_back, 0])
X, y = np.array(X), np.array(y)

# Split data into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Reshape data to fit LSTM input shape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the normalized predictions and true values
y_pred_original = scaler.inverse_transform(y_pred)
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot the predictions and true values
plt.figure(figsize=(12, 6))
plt.plot(data['Date'][split_index + look_back:], y_test_original, label='True Prices')
plt.plot(data['Date'][split_index + look_back:], y_pred_original, label='Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
